In [1]:
import requests
import pandas as pd
from web3 import Web3
from web3.contract import Contract
import json
import time

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
url = "https://api.chainbase.online/v1/dw/query"
headers = {
    "x-api-key": "..."
}
web3 = Web3(Web3.HTTPProvider('https://eth.llamarpc.com'))

In [4]:
# 通过 Transaction 接口获得交易哈希（这里会漏掉 Metamask 等 to_address的 OpenOcean 的合约）

payload_tx = {"query":"\r\nSELECT DISTINCT\r\n    transaction_hash\r\nFROM\r\n    ethereum.transactions\r\nWHERE\r\n    to_address = '0x6352a56caadc4f1e25cd6c75970fa768a3304e64'\r\nORDER BY\r\n    block_number DESC\r\nLIMIT\r\n    10\r\n\r\n\r\n\r\n\r\n"}
response_tx_hash = requests.post(url, json=payload_tx, headers=headers).json()
tx_hash_list = response_tx_hash['data']['result']

In [5]:
# 测试用
# for tx_hash in tx_hash_list:
#     print(tx_hash)

In [5]:
# 通过 Trace_call 表获取数据（Transaction Hash 和 Input）
def get_trace_call(tx_hash):
    payload_trace_call = {"query":"\r\nSELECT\r\n    *\r\nFROM\r\n    ethereum.trace_calls\r\nWHERE\r\n    to_address = '0x6352a56caadc4f1e25cd6c75970fa768a3304e64' AND transaction_hash = '%s'\r\n\r\n\r\n\r\n\r\n" % tx_hash}
    response_trace_call = requests.post(url, json=payload_trace_call, headers=headers).json()['data']['result']
    print(tx_hash)
    return response_trace_call
    
# 通过 Logs 表获取 Logs
def get_logs(tx_hash):
    payload_logs = {"query":"\r\nSELECT\r\n    *\r\nFROM\r\n    ethereum.transaction_logs\r\nWHERE\r\n    transaction_hash = '%s' \r\n" % tx_hash}
    response_logs = requests.post(url, json=payload_logs, headers=headers).json()['data']['result']
    return response_logs

In [6]:
# 创建 Transaction 表
df_transaction = pd.DataFrame(columns=['tx_hash', 'method', 'in_token_address', 'in_token_amount', 'out_token_address', 'out_token_amount', 'error_code'])

In [7]:
# 这里是等等要使用 web3.py 解析 swap function 的 input
abi_data = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"contract IERC20","name":"srcToken","type":"address"},{"indexed":true,"internalType":"contract IERC20","name":"dstToken","type":"address"},{"indexed":false,"internalType":"address","name":"dstReceiver","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"spentAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"returnAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"minReturnAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"guaranteedAmount","type":"uint256"},{"indexed":false,"internalType":"address","name":"referrer","type":"address"}],"name":"Swapped","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"}],"name":"callUniswap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"","type":"bytes32[]"},{"internalType":"address payable","name":"recipient","type":"address"}],"name":"callUniswapTo","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"},{"internalType":"bytes","name":"permit","type":"bytes"},{"internalType":"address payable","name":"recipient","type":"address"}],"name":"callUniswapToWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"},{"internalType":"bytes","name":"permit","type":"bytes"}],"name":"callUniswapWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"rescueFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IOpenOceanCaller","name":"caller","type":"address"},{"components":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"contract IERC20","name":"dstToken","type":"address"},{"internalType":"address","name":"srcReceiver","type":"address"},{"internalType":"address","name":"dstReceiver","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturnAmount","type":"uint256"},{"internalType":"uint256","name":"guaranteedAmount","type":"uint256"},{"internalType":"uint256","name":"flags","type":"uint256"},{"internalType":"address","name":"referrer","type":"address"},{"internalType":"bytes","name":"permit","type":"bytes"}],"internalType":"struct OpenOceanExchange.SwapDescription","name":"desc","type":"tuple"},{"components":[{"internalType":"uint256","name":"target","type":"uint256"},{"internalType":"uint256","name":"gasLimit","type":"uint256"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"internalType":"struct IOpenOceanCaller.CallDescription[]","name":"calls","type":"tuple[]"}],"name":"swap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"}],"name":"uniswapV3Swap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"int256","name":"amount0Delta","type":"int256"},{"internalType":"int256","name":"amount1Delta","type":"int256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"uniswapV3SwapCallback","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address payable","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"}],"name":"uniswapV3SwapTo","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"recipient","type":"address"},{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"},{"internalType":"bytes","name":"permit","type":"bytes"}],"name":"uniswapV3SwapToWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
abi = json.loads(abi_data)


for tx_hash_dict in tx_hash_list:
    tx_hash = tx_hash_dict['transaction_hash']
    
    # 避免超时（其实可以把 SQL 写好一点）
    time.sleep(2)
    
    trace_call = get_trace_call(tx_hash)
    
    # 判断是否交易失败，通过 trace_call 返回的 error 字段
    if trace_call[0]['error'] == 'Reverted':
        
        # 使用 web3.to_text 方法解析错误原因（在138之后的是可解析内容，前 10 个字节分别是 0x + 方法？）
        error_code = web3.to_text(trace_call[0]['output'][138:]) 
        tmp = {'tx_hash':tx_hash, 'method':0,'in_token':0, 'in_token_amount':0, 'out_token':0, 'out_token_amount':0, 'error_code':error_code}
        df_transaction.loc[len(df_transaction)]=tmp

    # 判断是否为 Univ3 方法，主要通过 trace_call 数量和 trace_call[0] 是否为 bc80f1a8 方法(UniswapV2 Trace 也可能 > 0)
    elif len(trace_call) > 1 and '0xbc80f1a8' in trace_call[0]['input']:
        for i in trace_call:
            if '0xbc80f1a8' in i['input']:
                
                # 0xbc80f1a8 方法后在多个 64 个字节就是 in_token_amount_hash
                in_token_amount_hash = i['input'][74:138]
                # output 里面的内容是 out_amount_hash
                out_token_amount_hash = i['output']
                # 解析一下
                in_token_amount = int(in_token_amount_hash, 16)
                out_token_amount = int(out_token_amount_hash, 16)
            else:
                pass

        logs = get_logs(tx_hash)
        for i in logs:
            # Logs 中的 data 如果和 in_token_amount_hash 相同的话，就意味着对应的 Token 为 in_token
            if i['data'] == '0x' + in_token_amount_hash:
                in_token_address = i['address']
            # logs 中的 data 如果和 out_token_amount_hash 相同的话，就意味着对应的 token 为 out_token
            if i['data'] == out_token_amount_hash:
                out_token_address = i['address']

        tmp = {'tx_hash':tx_hash, 'method':'bc80f1a8', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount}
        df_transaction.loc[len(df_transaction)]=tmp
        time.sleep(2)
    
    # 判断是否为 Univ2 方法，主要通过 trace_call[0] 是否为 6b58f2f0 方法
    elif trace_call[0]['method_id'] == '6b58f2f0':
         # 通过 Logs 表获取数据（Logs）
        logs = get_logs(tx_hash)

        # 获取 In_Token & Amount（in_token_address 和 amount 写在一起了）
        input_data = trace_call[0]['input']
        # 这里是把前面的 0 截掉了，完整的位置是 [10:74]
        in_token_address = '0x' + str(input_data[34:74])
        # 解析的时候不需要截 0
        in_token_amount = int(input_data[74:138], 16)

        # 获取 Out_Token & Amount
        output = trace_call[0]['output']

        for i in logs:
            # output 里面的内容是 out_amount_hash，所以对应 logs 中的 address 就是 out_token_address
            if i['data'] == output:
                # logs 中的 data 如果和 out_token_amount_hash 相同的话，就意味着对应的 token 为 out_token
                out_token_address = i['address']
                
                # out_amount 其实可以写在上面（和 log 没关系），但是这样写看着可能好看点
                out_token_amount = int(output, 16)
                
                # 部分 log 数据会有一些小尾巴，需要直接 Pass 掉（避免报错）
                pass

        tmp = {'tx_hash':tx_hash, 'method':'6b58f2f0', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
        df_transaction.loc[len(df_transaction)]=tmp
        time.sleep(2)
        
    # 判断是否为 Swap 方法，主要通过 trace_call[0] 是否为 6b58f2f0 方法
    elif trace_call[0]['method_id'] == '90411a32':
        #print('yes')
        
        # 使用代理合约 ABI 解析 input
        openocean_contract = web3.eth.contract('0x01Ec93c289cB003e741f621cDD4FE837243f8905', abi=abi)
        openocean_contract_decode = openocean_contract.decode_function_input(trace_call[0]['input'])
        
        # 解析完成之后填数字就行了
        in_token_address = openocean_contract_decode[1]['desc']['srcToken']
        in_token_amount = openocean_contract_decode[1]['desc']['amount']
        out_token_address = openocean_contract_decode[1]['desc']['dstToken']
        out_token_amount = int(trace_call[0]['output'], 16)

        tmp = {'tx_hash':tx_hash, 'method':'90411a32', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
        df_transaction.loc[len(df_transaction)]=tmp
        time.sleep(2)
        
    else:
        # 如果出现了不符合上述情况的内容，输出的表数据会重复出现，用来排查错误(Method 也改了一下名字)
        tmp = {'tx_hash':tx_hash, 'method':'Data_Error', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
        df_transaction.loc[len(df_transaction)]=tmp
        time.sleep(2)

0xe91c72ae2ccdc1a292cd27dfbee6ae8a6cd74119c5701b13e22c0d764b2f248f
0xcb863ab9da367c94b76e0b4b1644cfb31168d5745f4a6c2ab806358fec990978
0xe45ce8b162d6f6ab93f5d1ebf18925e9cf10b333ce7a0d94ff6782dce233366a
0x84ed4c8894c45067017388aad0ffa95f34b6d44361e0490fc39cba5af8d69f8d
0x6d687b593ca452da2e8a4b896741d8b90be63c0689a3d84a048b6230a749000d
0x69b08fbc4bd694444a8e06e0aa213b4faa0a70632bbf2999ebb14d349205b205
0xa5ed945462e58c77ad8e82e3afe612494c9e8d42c20aaf20ff12e120ed902712
0x7893b8c065738e73ff41f9c5e9b8999aa3b9686806d5719e0213db2c9a8a3511
0x4b03a4bef9318a4162f6ea50b6195343a8bc35a84457557293caaabad9c4b6ed
0x9e56265bc9f43d4bbbd3557e5010fd7d7b8ea5e2a9dd326d5d1398ec538d24bb


In [8]:
df_transaction

,tx_hash,method,in_token_address,in_token_amount,out_token_address,out_token_amount,error_code
0,0xe91c72ae2ccdc1a292cd27dfbee6ae8a6cd74119c570...,90411a32,0x5283D291DBCF85356A21bA090E6db59121208b44,65698300003928750000,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,7676379651196682,0
1,0xcb863ab9da367c94b76e0b4b1644cfb31168d5745f4a...,90411a32,0x38E68A37E401F7271568CecaAc63c6B1e19130B4,237552600000000000000,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,1207418755975736410,0
2,0xe45ce8b162d6f6ab93f5d1ebf18925e9cf10b333ce7a...,90411a32,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,6850503,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,4104085151185055,0
3,0x84ed4c8894c45067017388aad0ffa95f34b6d44361e0...,90411a32,0xdAC17F958D2ee523a2206206994597C13D831ec7,30196041,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,18098713254201541,0
4,0x6d687b593ca452da2e8a4b896741d8b90be63c0689a3...,90411a32,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,100000000000000000,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,100000000000000000,0
5,0x69b08fbc4bd694444a8e06e0aa213b4faa0a70632bbf...,90411a32,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,4200000000000000,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,6850503,0
6,0xa5ed945462e58c77ad8e82e3afe612494c9e8d42c20a...,90411a32,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,320000000000000000,0x644192291cc835A93d6330b24EA5f5FEdD0eEF9e,10240954109065103586164,0
7,0x7893b8c065738e73ff41f9c5e9b8999aa3b9686806d5...,90411a32,0xdAC17F958D2ee523a2206206994597C13D831ec7,10000000000,0xAf5191B0De278C7286d6C7CC6ab6BB8A73bA2Cd6,22162701654805005699752,0
8,0x4b03a4bef9318a4162f6ea50b6195343a8bc35a84457...,90411a32,0x38029C62DfA30D9FD3CaDf4C64e9b2ab21DbDa17,300000000000000000000,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,424532809147812328,0
9,0x9e56265bc9f43d4bbbd3557e5010fd7d7b8ea5e2a9dd...,90411a32,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,184000000000000000,0x120a3879da835A5aF037bB2d1456beBd6B54d4bA,4832931125715045429234,0


In [ ]:
# 单笔测试代码

In [34]:
# 创建 Transaction 表
df_transaction_test = pd.DataFrame(columns=['tx_hash', 'method', 'in_token_address', 'in_token_amount', 'out_token_address', 'out_token_amount', 'error_code'])

# 在这里输入需要被测试的哈希
tx_hash = r'0x9888de165e184d6f4af2f896277f1b36606c4b7110a85abdeabc985825bdb1dd'

abi_data = '[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"contract IERC20","name":"srcToken","type":"address"},{"indexed":true,"internalType":"contract IERC20","name":"dstToken","type":"address"},{"indexed":false,"internalType":"address","name":"dstReceiver","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"spentAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"returnAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"minReturnAmount","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"guaranteedAmount","type":"uint256"},{"indexed":false,"internalType":"address","name":"referrer","type":"address"}],"name":"Swapped","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"}],"name":"callUniswap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"","type":"bytes32[]"},{"internalType":"address payable","name":"recipient","type":"address"}],"name":"callUniswapTo","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"},{"internalType":"bytes","name":"permit","type":"bytes"},{"internalType":"address payable","name":"recipient","type":"address"}],"name":"callUniswapToWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"bytes32[]","name":"pools","type":"bytes32[]"},{"internalType":"bytes","name":"permit","type":"bytes"}],"name":"callUniswapWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IERC20","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"rescueFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IOpenOceanCaller","name":"caller","type":"address"},{"components":[{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"contract IERC20","name":"dstToken","type":"address"},{"internalType":"address","name":"srcReceiver","type":"address"},{"internalType":"address","name":"dstReceiver","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturnAmount","type":"uint256"},{"internalType":"uint256","name":"guaranteedAmount","type":"uint256"},{"internalType":"uint256","name":"flags","type":"uint256"},{"internalType":"address","name":"referrer","type":"address"},{"internalType":"bytes","name":"permit","type":"bytes"}],"internalType":"struct OpenOceanExchange.SwapDescription","name":"desc","type":"tuple"},{"components":[{"internalType":"uint256","name":"target","type":"uint256"},{"internalType":"uint256","name":"gasLimit","type":"uint256"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"internalType":"struct IOpenOceanCaller.CallDescription[]","name":"calls","type":"tuple[]"}],"name":"swap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"}],"name":"uniswapV3Swap","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"int256","name":"amount0Delta","type":"int256"},{"internalType":"int256","name":"amount1Delta","type":"int256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"uniswapV3SwapCallback","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address payable","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"}],"name":"uniswapV3SwapTo","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"recipient","type":"address"},{"internalType":"contract IERC20","name":"srcToken","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"minReturn","type":"uint256"},{"internalType":"uint256[]","name":"pools","type":"uint256[]"},{"internalType":"bytes","name":"permit","type":"bytes"}],"name":"uniswapV3SwapToWithPermit","outputs":[{"internalType":"uint256","name":"returnAmount","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
abi = json.loads(abi_data)

#time.sleep(2)

trace_call = get_trace_call(tx_hash)
if trace_call[0]['error'] == 'Reverted':
    error_code = web3.to_text(trace_call[0]['output'][138:]) 
    tmp = {'tx_hash':tx_hash, 'method':0,'in_token':0, 'in_token_amount':0, 'out_token':0, 'out_token_amount':0, 'error_code':error_code}
    df_transaction_test.loc[len(df_transaction_test)]=tmp

# 判断是否为 Univ3 方法(UniswapV2 Trace 也可能 > 0)
elif len(trace_call) > 1 and '0xbc80f1a8' in trace_call[0]['input']:
    for i in trace_call:
        if '0xbc80f1a8' in i['input']:
            in_token_amount_hash = i['input'][74:138]
            out_token_amount_hash = i['output']
            in_token_amount = int(i['input'][74:138], 16)
            out_token_amount = int(i['output'], 16)
        else:
            pass

    logs = get_logs(tx_hash)
    for i in logs:
        if i['data'] == '0x' + in_token_amount_hash:
            in_token_address = i['address']
        if i['data'] == out_token_amount_hash:
            out_token_address = i['address']

    tmp = {'tx_hash':tx_hash, 'method':'bc80f1a8', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount}
    df_transaction_test.loc[len(df_transaction_test)]=tmp
    #time.sleep(2)

elif trace_call[0]['method_id'] == '6b58f2f0':
     # 通过 Logs 表获取数据（Logs）
    logs = get_logs(tx_hash)

    # 获取 In_Token & Amount
    input_data = trace_call[0]['input']
    in_token_address = '0x' + str(input_data[34:74])
    in_token_amount = int(input_data[74:138], 16)

    # 获取 Out_Token & Amount
    output = trace_call[0]['output']

    for i in logs:
        if i['data'] == output:
            out_token_address = i['address']
            out_token_amount = int(output, 16)

            # 部分数据这里会有一些小尾巴，需要直接 Pass 掉
            pass

    tmp = {'tx_hash':tx_hash, 'method':'6b58f2f0', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
    df_transaction_test.loc[len(df_transaction_test)]=tmp
    #time.sleep(2)

# Swap 方法（使用代理合约 ABI 解析，但其实也可以用之前的方法一样做，但懒得搞了）
elif trace_call[0]['method_id'] == '90411a32':
    #print('yes')
    openocean_contract = web3.eth.contract('0x01Ec93c289cB003e741f621cDD4FE837243f8905', abi=abi)
    openocean_contract_decode = openocean_contract.decode_function_input(trace_call[0]['input'])
    in_token_address = openocean_contract_decode[1]['desc']['srcToken']
    in_token_amount = openocean_contract_decode[1]['desc']['amount']
    out_token_address = openocean_contract_decode[1]['desc']['dstToken']
    out_token_amount = int(trace_call[0]['output'], 16)

    tmp = {'tx_hash':tx_hash, 'method':'90411a32', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
    df_transaction_test.loc[len(df_transaction_test)]=tmp
    #time.sleep(2)

else:
    tmp = {'tx_hash':tx_hash, 'method':'SAME', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
    df_transaction_test.loc[len(df_transaction_test)]=tmp
    #time.sleep(2)
    
df_transaction_test

0x9888de165e184d6f4af2f896277f1b36606c4b7110a85abdeabc985825bdb1dd


,tx_hash,method,in_token_address,in_token_amount,out_token_address,out_token_amount,error_code
0,0x9888de165e184d6f4af2f896277f1b36606c4b7110a8...,90411a32,0xdAC17F958D2ee523a2206206994597C13D831ec7,8000000000,0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39,12612574309024,0


In [ ]:
# 简单测试总结
## CallUniswapV3
### UniswapV3 - 单路径 - 成功
#### 0x71d58cb2f5b57b7cf858e41f9471bf3a6cffc29013e8af0f7f8bc46c099b8bb5
### UniswapV3 - ETH 转 WETH - 成功(无法识别 ETH)
#### 0xc6c08a15f336618511b0367fefbac69954a4558506b32eade0a0b46ede14617b
### UniswapV3 多个路径 - 成功
#### 0x1bcd8a7867dab1e086307051edb7f99f5e385bb63093c8994fc3e313bb120009

## CallUniswapV2
### UniswapV2 - 单路径 - 成功
#### 0x40fa108224dfaee53f79170d65b929be47077f90425f88676b14985dbc7db484
### UniswapV2 - ETH 转 WETH - 成功（无法识别 ETH 以及 TAX）
#### 0xbd6562253a213046ae99fb4ea3c6cc41fbc12b22cd02c823c41b4aac9103bafe
### UniswapV2 - 多路径 - 成功
#### 0x8c9cf33f61a6031341b07277a776c511b42bae2719cd5b5392f81f05b4fab050

## Swap
### swap - 单路径 - 成功
#### 0xd67bdee696b7153081dc91fe1544f9a3da205cfdceb8638dff579606a1841fcb
### swap - ETH 转 WETH - 成功（无法识别 WETH 以及 TAX）
#### 0x4b730304d2ccb31354fa6177df33cde278495ec908f26037aedc88afb6591e92
### swap - 多路径 - 成功
#### 0x4f2c41afd7618746463590fec01ebd3002d2e877028fd9b4c2078a8ad43cf7a3
#### 0x9888de165e184d6f4af2f896277f1b36606c4b7110a85abdeabc985825bdb1dd

## 问题
### Transaction Hash 有点不全，目前是使用 to_address 获取的